In [16]:
import requests
import os

def audio_request(input_file, url, return_json=True):
    if not os.path.isfile(input_file):
        print(f"Error: File {input_file} does not exist.")
        return None
    
    with open(input_file, 'rb') as file:
        files = {'audio': file}
        try:
            response = requests.post(url, files=files)
            response.raise_for_status()
            return response.json() if return_json else response.text
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

def text_request(input_text, url):
    try:
        response = requests.post(url, json={'text': input_text})
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Usage example
BASE_DIR = "./standardized"
input_file = "first_ten_Sample_HV_Clip.wav"
full_path = os.path.join(BASE_DIR, input_file)

# Language identification from audio
lang_url = "http://localhost:8001/identify_language"
lang_result = audio_request(full_path, lang_url)
print(f"Detected language: {lang_result}" if lang_result else "Failed to detect language")

Detected language: {'language': 'en: English', 'likelihood': 0.9994450211524963}


In [17]:
# ASR
asr_url = "http://localhost:8001/transcribe"
asr_result = audio_request(full_path, asr_url, return_json=False)
print(f"ASR result: {asr_result}" if asr_result else "Failed to transcribe audio")

ASR result: NOW WE ARE RECORDING WE'RE GOING TO START OFF SOME BASIC DEMOCRATIC QUESTIONS WHAT IS YOUR CURRENT MYRTLE STATUS MARRIED NO YOU WRITE OR LEFT HANDED


In [18]:
if asr_result:
    # NLTK sentiment
    sentiment_url = "http://localhost:8000/sentiment"
    sentiment = text_request(asr_result, sentiment_url)
    if sentiment:
        print(f"NLTK sentiment: {sentiment}" if sentiment else "Failed to get sentiment")

    # NLTK POS tagging
    pos_url = "http://localhost:8000/pos_tag"
    pos = text_request(asr_result, pos_url)
    if pos:
        print(f"NLTK POS: {pos}" if pos else "Failed to get POS tags")

    # BRUNet index - NLP container
    brunet_url = "http://localhost:8002/brunet_index"
    brunet = text_request(asr_result, brunet_url)
    if brunet:
        print(f"Brunet index: {brunet}" if brunet else "Failed to get Brunet index")
        
    # Lemmatization - Spacy
    lemmatization_url = "http://localhost:8003/lemmatize"
    lemmatization = text_request(asr_result, lemmatization_url)
    if lemmatization:
        print(f"Lemmatization: {lemmatization}" if lemmatization else "Failed to get lemmatization")

NLTK sentiment: {'sentiment_scores': {'compound': -0.296, 'neg': 0.081, 'neu': 0.919, 'pos': 0.0}}
NLTK POS: {'tagged': [['NOW', 'IN'], ['WE', 'NNP'], ['ARE', 'NNP'], ['RECORDING', 'NNP'], ['WE', 'NNP'], ["'RE", 'VBD'], ['GOING', 'NNP'], ['TO', 'NNP'], ['START', 'NNP'], ['OFF', 'NNP'], ['SOME', 'NNP'], ['BASIC', 'NNP'], ['DEMOCRATIC', 'NNP'], ['QUESTIONS', 'NNP'], ['WHAT', 'NNP'], ['IS', 'VBZ'], ['YOUR', 'JJ'], ['CURRENT', 'NNP'], ['MYRTLE', 'NNP'], ['STATUS', 'NNP'], ['MARRIED', 'NNP'], ['NO', 'NNP'], ['YOU', 'NNP'], ['WRITE', 'NNP'], ['OR', 'NNP'], ['LEFT', 'NNP'], ['HANDED', 'NNP']]}
Brunet index: {'brunet_index': 6.790982785097943, 'total_words': 26, 'unique_words': 25}
Lemmatization: {'lemmas': ['now', 'we', 'be', 'record', "WE'RE", 'go', 'to', 'start', 'off', 'some', 'basic', 'democratic', 'question', 'what', 'be', 'your', 'current', 'MYRTLE', 'STATUS', 'MARRIED', 'no', 'you', 'write', 'or', 'LEFT', 'HANDED']}
